<a href="https://colab.research.google.com/github/tsilverio/Desafio1_DataScience/blob/master/Titanic_com_v%C3%A1rios_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Montando google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##Importando bibliotecas

In [0]:
import numpy as np
import pandas as pd

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error

np.random.seed(0)

##Criando funções para comparar diferentes métodos

###ExtraTreesClassifier

In [0]:
def compara_acuracia_ExtraTreesClassifier(max_leaf_nodes, train_X, valid_X, train_y, valid_y):
    model = ExtraTreesClassifier(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    predictions = model.predict(valid_X)
    mae = mean_absolute_error(valid_y, predictions)
    acc = np.mean(valid_y == predictions)    
    score = model.score(train_X, train_y)
    return(acc)

###RandomForestClassifier usar este

In [0]:
def compara_acuracia_RandomForestClassifier(max_leaf_nodes, train_X, valid_X, train_y, valid_y):
    model = RandomForestClassifier(n_estimators=100, max_depth=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    predictions = model.predict(valid_X)
    mae = mean_absolute_error(valid_y, predictions).astype(int)
    acc = np.mean(valid_y == predictions)
    score = model.score(train_X, train_y)
    return(acc)

###RandomForestRegressor

In [0]:
def compara_acuracia_RandomForestRegressor(max_leaf_nodes, train_X, valid_X, train_y, valid_y):
    model = RandomForestRegressor(n_estimators=100, max_depth=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    predictions = model.predict(valid_X)
    mae = mean_absolute_error(valid_y, predictions).astype(int)
    acc = np.mean(valid_y == predictions)
    score = model.score(train_X, train_y)
    return(acc)

###LogisticRegression

In [0]:
def compara_acuracia_LogisticRegression(max_leaf_nodes, train_X, valid_X, train_y, valid_y):
    model = LogisticRegression(max_iter=max_leaf_nodes)
    model.fit(train_X, train_y)
    predictions = model.predict(valid_X)
    mae = mean_absolute_error(valid_y, predictions)
    acc = np.mean(valid_y == predictions)
    score = model.score(train_X, train_y)
    return(acc)

## Importando os arquivos

In [0]:
train_data = pd.read_csv("/content/drive/My Drive/Titanic/train.csv")


In [0]:
test_data = pd.read_csv("/content/drive/My Drive/Titanic/test.csv")


###Verificando o dataframe

In [45]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Tamanho_nome
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24


In [0]:
test_data.head()

In [0]:
train_data["Survived"] = train_data["Survived"].astype(float) ##quando não estava conseguindo a acurácia com decimal

##Preenchendo valores nulos, criando e transformando variáveis

In [0]:
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].mean())
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mean())
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode())
train_data['Tamanho_nome'] = train_data['Name'].apply(len)

#train_data['Cabin'] = np.where(train_data["Cabin"].isnull(), 0, train_data['Cabin'])

test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
test_data['Embarked'] = test_data['Embarked'].fillna(test_data['Embarked'].mode())
test_data['Tamanho_nome'] = test_data['Name'].apply(len)

#test_data['Cabin'] = np.where(test_data["Cabin"].isnull(), 0, test_data['Cabin'])

## Setando as variáveis dummies

In [0]:
variaveis = ["Pclass", "Sex", "SibSp", "Parch","Embarked"]
#variaveis = ["Pclass", "Sex", "SibSp", "Parch","Embarked","Cabin"]

##Populando o X e o y de treino

In [0]:
y = train_data["Survived"]
X = pd.get_dummies(train_data[variaveis])

X['Fare'] = train_data['Fare']
X['Age'] = train_data['Age']
X['Tamanho_nome'] = train_data['Tamanho_nome']

#X['Cabin'] = train_data["Cabin"]

###Validando o X

In [28]:
X.head(1)

,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Fare,Age,Tamanho_nome
0,3,1,0,0,1,0,0,1,7.25,22.0,23


## Populando o X de teste

In [0]:
X_test = pd.get_dummies(test_data[variaveis])

X_test['Fare'] = test_data['Fare']
X_test['Age'] = test_data['Age']
X_test['Tamanho_nome'] = test_data['Tamanho_nome']

#X_test['Cabin'] = test_data["Cabin"]

## Separando o dataset de treino e validação

In [0]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y, random_state = 0)

###Validando

In [30]:
train_X.shape

(668, 11)

In [0]:
train_y.shape

(668,)

In [0]:
valid_X.shape

(223, 11)

In [0]:
valid_y.shape

(223,)

##Rodando o modelo com dados de treino e prevendo com dados de validação

###Única execução

In [0]:
model = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=1)
model.fit(train_X, train_y)
predictions = model.predict(valid_X)

np.mean(valid_y == predictions)

0.8385650224215246

###ExtraTreesClassifier

In [43]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    retorno = compara_acuracia_ExtraTreesClassifier(max_leaf_nodes, train_X, valid_X, train_y, valid_y)
    #print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, retorno))
    print(retorno)

0.7802690582959642
0.8340807174887892
0.7937219730941704
0.7937219730941704


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


###RandomForestClassifier

In [44]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    retorno = compara_acuracia_RandomForestClassifier(max_leaf_nodes, train_X, valid_X, train_y, valid_y)
    print(retorno)

0.8071748878923767
0.8340807174887892
0.8340807174887892
0.8340807174887892


###RandomForestRegressor

In [0]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    retorno = compara_acuracia_RandomForestRegressor(max_leaf_nodes, train_X, valid_X, train_y, valid_y)
    print(retorno)

0.0
0.08071748878923767
0.08071748878923767
0.08071748878923767


###LogisticRegression

In [0]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    retorno = compara_acuracia_LogisticRegression(max_leaf_nodes, train_X, valid_X, train_y, valid_y)
    print(retorno)

0.7713004484304933
0.7982062780269058
0.7982062780269058
0.7982062780269058


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureW

##Prevendo o modelo com dados de teste

In [0]:
predictions = model.predict(X_test)

## Criar submission

In [0]:
sub = pd.Series(predictions, index=test_data['PassengerId'], name='Survived')
sub.to_csv("Sub RandomForest com sete variáveis split.csv", header=True)

In [0]:
!head -n10 "Sub RandomForest.csv"

head: cannot open 'Sub RandomForest.csv' for reading: No such file or directory
